<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


# All NLP Imports

import spacy
nlp = spacy.load('en_core_web_lg')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [3]:
# get rid of whitespace and also make lowercase

def wrangle(df):
    df = df.copy()
    df['text'] = df['text'].str.lower().str.strip()
    df['text'] = df['text'].str.replace(r'[^a-zA-Z ^0-9]', '')
    return df

yelp = wrangle(yelp)

In [4]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,beware fake fake fakewe also own a small busin...,10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,came here for lunch togo service was quick sta...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,ive been to vegas dozens of times and had neve...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,we went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,35 to 4 starsnot bad for the price 1299 for lu...,5,n9QO4ClYAS7h9fpQwa5bhA


In [5]:
yelp.shape

(10000, 9)

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [6]:
# basic preprocess using generated stopwords

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

STOPWORDS = set(STOPWORDS).union(set([' ', '  ', '   ']))

def tokenize(text):
    return [token for token in simple_preprocess(text, deacc=True, min_len=4, max_len=20) 
            if token not in STOPWORDS]

In [7]:
yelp['tokenized'] = yelp['text'].apply(tokenize)
yelp['tokenized'].head()

0    [beware, fake, fake, fakewe, small, business, ...
1    [came, lunch, togo, service, quick, staff, fri...
2    [vegas, dozens, times, stepped, foot, circus, ...
3    [went, night, closed, street, party, best, act...
4    [starsnot, price, lunch, seniors, eating, food...
Name: tokenized, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [8]:
# vector representation

vect = TfidfVectorizer(tokenizer=tokenize)

dtm = vect.fit_transform(yelp['text'])
dtm_df = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())
dtm_df.head()

,aaaahhhs,aaasssk,aabs,aamco,aand,aaron,aback,abandoned,abby,abdc,...,zumanity,zumba,zuni,zupas,zuzana,zuzu,zxkxko,zyrtec,zzaplon,zzzzzzzzzmy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# import nearest neighbors
from sklearn.neighbors import NearestNeighbors

# run model

nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm_df)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [10]:
fake_review = ["This is the worst restaurant I have ever been to. Who even likes asian fusion? I am not sure I will ever come back here. But the service was good."]

fake_review_transform = vect.transform(fake_review)

In [11]:
neighbors = nn.kneighbors(fake_review_transform.todense())
neighbors

(array([[1.        , 1.        , 1.15819679, 1.20227059, 1.22170597]]),
 array([[6311, 6204, 5474, 6099, 1522]]))

In [12]:
# look at some of them

yelp['text'][5474]

'asian fusion at its best it was amazing i wish the menu and or yelp had more pictures to see what the dishes looked like too hard to decide when you dont know the dishes service was great'

In [13]:
yelp['text'][6099]

'it was our first time trying out asian fusion i was delighted with the decoration on the walls the servers was good and really attentive the food was tasty and good portion there was a wide variety of cuisines on the menu so lots of choices we liked it a lot '

### Ok seems like its finding asian fusion from my fake review. Cool!

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [14]:
# imports for pipeline

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

# variables

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

# pipeline
pipe = Pipeline([('vect', vect), ('clf', clf)])

# params

parameters = {
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

# search and fit

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(yelp['text'], yelp['stars'])

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:   59.6s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [16]:
# I had to use super simple params in order to get this to work, kernel kept crashing!! so frustrating
grid_search.best_params_

{'clf__max_depth': 20, 'vect__max_df': 0.75}

In [18]:
# prediction on fake review

pred = grid_search.predict(fake_review)

# blanking on how to get prediction out!

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [19]:
from gensim import corpora
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

In [20]:
# create generator
def get_reviews(df, token_col):
    assert token_col in df.columns, f"{token_col} does not exist!"
    
    for index, row in df.iterrows():
        review_token = row[token_col]
        yield review_token

Learn the vocubalary of the yelp data:

In [21]:
id2word = corpora.Dictionary(get_reviews(yelp, 'tokenized'))

Create a bag of words representation of the entire corpus

In [22]:
corpus = [id2word.doc2bow(review) for review in get_reviews(yelp, 'tokenized')]

Your LDA model should be ready for estimation: 

In [23]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 5 # You can change this parameter
                  )

In [25]:
import re

def print_lda_topics(lda):
    words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
    topics = [' '.join(t[0:5]) for t in words]
    for id, t in enumerate(topics): 
        print(f"------ Topic {id} ------")
        print(t, end="\n\n")
print_lda_topics(lda)

------ Topic 0 ------
food like great place good

------ Topic 1 ------
place service food great good

------ Topic 2 ------
good place food service time

------ Topic 3 ------
place good food great service

------ Topic 4 ------
great good place food like



Create 1-2 visualizations of the results

In [26]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [27]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.001255  0.003242       1        1  20.624641
3      0.004679  0.000985       2        1  20.368250
1      0.001379 -0.004466       3        1  19.946627
2     -0.000306  0.001596       4        1  19.654518
0     -0.004498 -0.001357       5        1  19.405973, topic_info=           Term         Freq        Total Category  logprob  loglift
125       place  4379.000000  4379.000000  Default  30.0000  30.0000
38         good  4217.000000  4217.000000  Default  29.0000  29.0000
43      service  3269.000000  3269.000000  Default  28.0000  28.0000
332  definitely  1335.000000  1335.000000  Default  27.0000  27.0000
44        staff  1439.000000  1439.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
605  restaurant   230.980515  1256.061279   Topic5  -5.9159  -0.0538
186        dont   275.754730  1787.279663   Topic5  -5.7387  -0.2293
443        know   215.253418  1137.309326   Topic5  -5.9864  -0.0250
114        best   242.754486  1684.107056   Topic5  -5.8662  -0.2974
44        staff   224.231827  1439.041870   Topic5  -5.9455  -0.2195

[475 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
20694      1  0.173581  abordable
20694      2  0.173581  abordable
20694      3  0.347161  abordable
20694      4  0.173581  abordable
20694      5  0.173581  abordable
...      ...       ...        ...
295        1  0.155321      youre
295        2  0.203754      youre
295        3  0.155321      youre
295        4  0.247178      youre
295        5  0.237157      youre

[1378 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 3, 1])

### So I noticed using the gensim visualization as well as just from topic text that my results seem to correlate around the same kind of words (obviously, but good sign that I did it correctly!) words like good, great, place, service, excellent, etc tend to congregate and seem to have about a ~20% chance of appearing in any given review.

### Another thing I noticed is that the word chicken seems to vary wildly as far as placement, but is definitely always in the mix. Everyone loves chicken! I wish I had time for the stretch goals. I could definitely knock one or two of them out. Stupid kernel kept crashing for an hour! And I am out of time. But I loved this unit!

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)